In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import torch
from collections import defaultdict
from google.colab.patches import cv2_imshow
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd


In [ ]:
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)


In [ ]:
from PIL import Image
# root_dir = '../input/output-crns/CRNS_OUTPUT'
class HeightandWidth(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)  # Chargement des données à partir d'un fichier CSV
        self.root_dir = root_dir  # Répertoire racine contenant les images
        self.transform = transform  # Transformation à appliquer aux images

    def __len__(self):
        return len(self.data)  # Retourne la taille du dataset

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])  # Chemin d'accès à l'image
        print(img_name)
        image = cv2.imread(img_name)  # Lecture de l'image à l'aide de OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Conversion de l'espace de couleur de BGR à RGB

        label = self.data.iloc[idx, 1]  # Extraction de l'étiquette associée à l'image

        if self.transform:
            pic = self.transform(image)  # Application de la transformation spécifiée aux images

        return pic, label


In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convertir l'image en objet de type PIL (Pillow)
    transforms.Resize((900, 700)),
    transforms.RandomRotation(10),  # Effectuer une rotation aléatoire de l'image dans la plage de -10 à +10 degrés
    transforms.ToTensor()  # Convertir l'image en tenseur PyTorch
])
#transforms.Resize((900, 700)),  # Redimensionner l'image à une taille spécifiée

In [ ]:
def train_epoch(model, train_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    real_values = []  # To store real values
    predicted_values = []
    for inputs, labels in train_loader:
        #print("\n labels",labels)
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels.unsqueeze_(dim=1)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        #print("label float(train)",labels.float(),"outputs",outputs)
        loss = loss_fn(outputs, labels.float())
        real_values.extend(labels.float().tolist())
        predicted_values.extend(outputs.tolist())
        losses.append(loss.item())
       # Calcul des gradients, mise à jour des poids et réinitialisation des gradients à zéro
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    threshold = 0.25
    matches = (np.array(predicted_values) >= (np.array(real_values) - threshold)) & (np.array(predicted_values) <= (np.array(real_values) + threshold))
    matches_tensor = torch.from_numpy(matches.astype(int))
    correct_predictions += torch.sum(matches_tensor)
        # Calcul des gradients, mise à jour des poids et réinitialisation des gradients à zéro

    plt.scatter(real_values, predicted_values)
    plt.xlabel('Real Values')
    plt.ylabel('Predicted Values')
    plt.title('Real Values vs Predicted Values(train)')
    plt.show()
    # Mise à jour du scheduler
    scheduler.step()

    # Calcul de la précision et de la perte moyenne
    return float(correct_predictions) / len(predicted_values), np.mean(losses)


In [ ]:
def eval_model(model, validation_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    real_values = []  # To store real values
    predicted_values = []

    for inputs, labels in validation_loader:

        inputs = inputs.to(device)
        #print("inputs :",inputs)
        labels = labels.to(device)
        labels.unsqueeze_(dim=1)
        outputs = model(inputs)

        #print('label_float(eval)',labels.float(),'  outputs  ',outputs)
        # Calcul des prédictions et de la perte
        _, preds = torch.max(outputs, 1)
        loss = loss_fn(outputs, labels.float())
        real_values.extend(labels.float().tolist())
        predicted_values.extend(outputs.tolist())
        losses.append(loss.item())
        # Mise à jour des prédictions correctes et des pertes
    threshold = 0.25
    matches = (np.array(predicted_values) >= (np.array(real_values) - threshold)) & (np.array(predicted_values) <= (np.array(real_values) + threshold))
    matches_tensor = torch.from_numpy(matches.astype(int))
    correct_predictions += torch.sum(matches_tensor)
    plt.scatter(real_values, predicted_values)

    plt.xlabel('Real Values')
    plt.ylabel('Predicted Values')
    plt.title('Real Values vs Predicted Values(val)')
    plt.show()
    # Calcul de la précision et de la perte moyenne
    return float(correct_predictions) / len(predicted_values), np.mean(losses)


In [ ]:
def train_model(model, device, n_epochs=60):
    dataset = HeightandWidth(csv_file='/content/drive/MyDrive/olive/Tree_height.csv',
                          root_dir='/content/drive/MyDrive/olive/segmented/',
                          transform=transform)

    batch_size = 5
    validation_split = 0.2
    shuffle_dataset = True
    random_seed = 42

    # Obtention de la taille du jeu de données
    dataset_size = len(dataset)
    print("\n dataset_size",dataset_size)
    # Création de la liste des indices des échantillons
    indices = list(range(dataset_size))
    print("\n indices",indices)
    # Calcul de l'indice de séparation entre les ensembles d'entraînement et de validation
    split = int(np.floor(validation_split * dataset_size))
    print("\n split",split)

    # Mélange aléatoire des indices si shuffle_dataset est True
    if shuffle_dataset:
      np.random.seed(random_seed)
      np.random.shuffle(indices)

    # Division des indices en ensembles d'entraînement et de validation
    train_indices, val_indices = indices[split:], indices[:split]
    print("\n train_indices",train_indices)
    print("\n val_indices",val_indices)

    # Création des échantillonneurs pour les ensembles d'entraînement et de validation
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    print("\n train_sampler",train_sampler)
    print("\n valid_sampler",valid_sampler)

    # Création des chargeurs de données pour l'entraînement et la validation
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

    # Set requires_grad=False for all parameters initially
    save_dir = '/content/drive/MyDrive/olive/'
    #weight_decay = 0.001
    weight_decay = 0
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.6,weight_decay=weight_decay)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    loss_fn = nn.MSELoss()
    history = defaultdict(list)
    best_accuracy = 0
    best_train_loss=0
    best_test_loss=0
    for epoch in range(n_epochs):
        print(f'Epoch {epoch + 1}/{n_epochs}')
        print('-' * 10)
        #print("train loader" ,len(train_loader))

        # Entraînement du modèle
        train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device, scheduler, len(train_loader))
        print(f'Train loss {train_loss}')
        print(f'train acc {train_acc}')
        # Évaluation du modèle sur les données de validation
        val_acc, val_loss = eval_model(model, validation_loader, loss_fn, device, len(validation_loader))
        print(f'Val loss {val_loss}')
        print(f'val acc {val_acc}')
        #print()
        torch.save(model.state_dict(), save_dir + 'resnet50_18_07_checkpointx.pth')
        # Enregistrement des métriques d'entraînement et de validation dans l'historique
        history['train_acc'].append(train_acc)
        history['train_loss'].append(train_loss)
        history['val_acc'].append(val_acc)
        history['val_loss'].append(val_loss)

        # Sauvegarde du meilleur modèle basé sur la précision de validation
        if val_acc > best_accuracy:
            torch.save(model.state_dict(), 'best_model_state.bin')
            best_accuracy = val_acc
            best_train_loss=train_loss
            best_test_loss = val_loss
    epochs = range(1, len(history['train_loss']) + 1)

    plt.plot(epochs, history['train_loss'], 'b', label='Training Loss')
    plt.plot(epochs, history['val_loss'], 'r', label='Validation Loss')
    plt.title('Training Loss vs. Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.plot(epochs, history['train_acc'], 'b', label='Training Accuracy')
    plt.plot(epochs, history['val_acc'], 'r', label='Validation Accuracy')
    plt.title('Training Accuracy vs. Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    print( "best_accuracy: ", best_accuracy)
    print("best_train_loss: ",best_train_loss)
    print("best_test_loss: ",best_test_loss)


    return model, history


In [ ]:
model50 = models.resnet50(pretrained=True)
print(model50.fc)

In [ ]:
num_features = model50.fc.in_features

print("Number of features:", num_features)

In [ ]:

for param in model50.parameters():
    # entrainer que final layer
    param.required_grad = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model50.to(device)
dropout_prob = 0  # Set the dropout probability

model50.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(inplace=True),
    nn.Dropout(dropout_prob),  # Add dropout layer here
    nn.Linear(1024, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(dropout_prob),  # Add dropout layer here
    nn.Linear(512, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(dropout_prob),  # Add dropout layer here
    nn.Linear(128, 64),
    nn.ReLU(inplace=True),
    nn.Dropout(dropout_prob),  # Add dropout layer here
    nn.Linear(64, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(dropout_prob),  # Add dropout layer here
    nn.Linear(128, 1)
).to(device)


In [ ]:
from torchsummary import summary
# Print the summary of the model
summary(model50,input_size=(3,900, 700))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model50, history50 = train_model(model50, device)
